In [1]:
import sys
sys.path.append("models/")

import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.stats import entropy
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GroupKFold
import statsmodels.api as sm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from dataloader import SurveyDataset, load_demo_v1, load_aggregate_travel_behavior, train_test_split_data
import mnl
import linear_reg
from setup import *

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# target variables y
data_version = '1571'

file = "origin_trip_behavior.csv"
df_pivot = load_aggregate_travel_behavior(file, data_version)
variable_names = ['active','auto','mas','pt','trpgen']

train_test_index = df_pivot['train_test'].astype(bool)
y = df_pivot[variable_names].to_numpy()[:,:4]



In [17]:
df_pivot[df_pivot['geoid'].isin(['17_31_842100','17_31_802300','17_31_770500','17_31_804201','17_31_20702','17_31_839100'])]


,state_fips_1,county_fips_1,tract_fips_1,active,auto,mas,pt,trpgen,geoid,area,state_fips,county_fips,tract_fips,latitude,longitude,train_test
254,17,31,20702,0.065261,0.769593,0.031023,0.134123,10.153113,17_31_20702,0.703820,17,31,20702,41.994126,-87.704894,2
608,17,31,770500,0.000000,0.935474,0.047098,0.017428,5.896687,17_31_770500,6.749911,17,31,770500,42.008006,-87.944741,3
646,17,31,802300,0.065440,0.865989,0.017926,0.050644,27.753065,17_31_802300,5.213576,17,31,802300,42.090458,-87.824237,1
692,17,31,804201,0.000000,0.992031,0.005454,0.002515,40.865249,17_31_804201,79.144406,17,31,804201,42.108269,-88.188316,0
1079,17,31,839100,0.383956,0.120298,0.028357,0.467389,369.319042,17_31_839100,1.185222,17,31,839100,41.880986,-87.633070,1
1099,17,31,842100,0.707839,0.028157,0.000000,0.264003,4.072322,17_31_842100,2.033199,17,31,842100,41.895279,-87.733610,1


In [29]:
df_pivot[df_pivot['geoid'].isin(['17_31_530200','17_31_610300'])]

,state_fips_1,county_fips_1,tract_fips_1,active,auto,mas,pt,trpgen,geoid,area,state_fips,county_fips,tract_fips,latitude,longitude,train_test
472,17,31,530200,0.146916,0.614424,0.00000,0.238661,3.746396,17_31_530200,1.296396,17,31,530200,41.681561,-87.632395,2
502,17,31,610300,0.051758,0.698611,0.09557,0.154061,5.990193,17_31_610300,2.381935,17,31,610300,41.816178,-87.675102,3


In [28]:
demo[demo['geoid'].isin(['17_31_530200','17_31_610300'])]

,geoid,tot_population,pct25_34yrs,pct35_50yrs,pctover65yrs,pctwhite_alone,pct_nonwhite,pctblack_alone,pct_col_grad,avg_tt_to_work,inc_per_capita,state_fips,county_fips,tract_fips,latitude,longitude,train_test
371,17_31_530200,0.029644187573967026,0.23091733485101837,0.3661989795918367,0.3007748261709659,0.0076530612244897,0.9923469387755102,0.9900510204081632,0.14010447990000116,0.7577570163052749,0.11705190866205167,17,31,530200,41.681561,-87.632395,2
393,17_31_610300,0.024452367886271333,0.1984626550683563,0.3893283958929473,0.10476795143025268,0.5185995623632386,0.4814004376367615,0.0361891937384278,0.11315314611696053,0.5882222630790993,0.07834447384152889,17,31,610300,41.816178,-87.675102,3


In [3]:
# explanatory variables x

demo_variables = ['tot_population','pct25_34yrs','pct35_50yrs','pctover65yrs',
         'pctwhite_alone', 'pct_nonwhite',
         'pctblack_alone',
         'pct_col_grad','avg_tt_to_work','inc_per_capita']

demo_cs, demo_np = load_demo_v1(data_dir, norm = 'minmax')
demo = np.hstack((np.array(demo_cs).reshape(-1,1), demo_np))

demo = pd.DataFrame(demo, columns = ['geoid'] + demo_variables)
demo = train_test_split_data(demo, data_version)

demo_variables = ['tot_population','pct25_34yrs','pct35_50yrs','pctover65yrs',
         'pctwhite_alone', #'pct_nonwhite',
         'pctblack_alone',
         'pct_col_grad','avg_tt_to_work','inc_per_capita']

x = demo[demo_variables].to_numpy(dtype=float)

groups = demo['train_test']

group_split = GroupKFold(n_splits=5)

In [10]:
x.mean(axis=0)

array([0.03459964, 0.27158335, 0.44796609, 0.2635742 , 0.67193107,
       0.15872199, 0.45013084, 0.61496312, 0.26206151])

# 1. Linear Regresion

### 1.1 Auto Share

In [88]:
y_index = 1

In [89]:
lr = linear_model.LinearRegression()
cross_results = cross_validate(lr, x, y[:,y_index], cv=group_split, groups=groups, scoring='r2', return_train_score=True)
print("Train score: %.4f \t Cross val score: %.4f \t " % 
          (cross_results['train_score'].mean(), cross_results['test_score'].mean()))

Train score: 0.5510 	 Cross val score: 0.5405 	 


In [81]:
# Lasso
for a in (1e-4)*np.array([0.1,1,2,3,4,5,6,7,8,10,20,50]):
    lasso = linear_model.Lasso(alpha=a)
    cross_results = cross_validate(lasso, x, y[:,y_index], cv=group_split, groups=groups, scoring='r2', return_train_score=True, return_estimator=True)
    nz = 0
    for m in cross_results['estimator']:
        nz += sum(m.coef_ != 0)
    nz /= 5
    
    print("Parameter: %.2e \t Train score: %.4f \t Cross val score: %.4f \t Nonzero coef: %d" % 
          (a, cross_results['train_score'].mean(), cross_results['test_score'].mean(), nz))

Parameter: 1.00e-05 	 Train score: 0.5510 	 Cross val score: 0.5405 	 Nonzero coef: 9
Parameter: 1.00e-04 	 Train score: 0.5508 	 Cross val score: 0.5407 	 Nonzero coef: 8
Parameter: 2.00e-04 	 Train score: 0.5501 	 Cross val score: 0.5403 	 Nonzero coef: 8
Parameter: 3.00e-04 	 Train score: 0.5491 	 Cross val score: 0.5392 	 Nonzero coef: 8
Parameter: 4.00e-04 	 Train score: 0.5477 	 Cross val score: 0.5377 	 Nonzero coef: 7
Parameter: 5.00e-04 	 Train score: 0.5460 	 Cross val score: 0.5360 	 Nonzero coef: 7
Parameter: 6.00e-04 	 Train score: 0.5440 	 Cross val score: 0.5342 	 Nonzero coef: 7
Parameter: 7.00e-04 	 Train score: 0.5417 	 Cross val score: 0.5322 	 Nonzero coef: 6
Parameter: 8.00e-04 	 Train score: 0.5392 	 Cross val score: 0.5302 	 Nonzero coef: 6
Parameter: 1.00e-03 	 Train score: 0.5335 	 Cross val score: 0.5252 	 Nonzero coef: 6
Parameter: 2.00e-03 	 Train score: 0.4928 	 Cross val score: 0.4852 	 Nonzero coef: 5
Parameter: 5.00e-03 	 Train score: 0.3857 	 Cross val 

### 1.2 PT

In [82]:
y_index = 3

In [87]:
lr = linear_model.LinearRegression()
cross_results = cross_validate(lr, x, y[:,y_index], cv=group_split, groups=groups, scoring='r2', return_train_score=True)
print("Train score: %.4f \t Cross val score: %.4f \t " % 
          (cross_results['train_score'].mean(), cross_results['test_score'].mean()))

Train score: 0.4313 	 Cross val score: 0.4166 	 


In [86]:
# Lasso
for a in (1e-4)*np.array([0.1,1,2,3,4,5,6,7,8,10,20,50]):
    lasso = linear_model.Lasso(alpha=a)
    cross_results = cross_validate(lasso, x, y[:,y_index], cv=group_split, groups=groups, scoring='r2', return_train_score=True, return_estimator=True)
    nz = 0
    for m in cross_results['estimator']:
        nz += sum(m.coef_ != 0)
    nz /= 5
    
    print("Parameter: %.2e \t Train score: %.4f \t Cross val score: %.4f \t Nonzero coef: %d" % 
          (a, cross_results['train_score'].mean(), cross_results['test_score'].mean(), nz))

Parameter: 1.00e-05 	 Train score: 0.4313 	 Cross val score: 0.4167 	 Nonzero coef: 8
Parameter: 1.00e-04 	 Train score: 0.4302 	 Cross val score: 0.4172 	 Nonzero coef: 7
Parameter: 2.00e-04 	 Train score: 0.4273 	 Cross val score: 0.4159 	 Nonzero coef: 6
Parameter: 3.00e-04 	 Train score: 0.4229 	 Cross val score: 0.4130 	 Nonzero coef: 6
Parameter: 4.00e-04 	 Train score: 0.4172 	 Cross val score: 0.4079 	 Nonzero coef: 6
Parameter: 5.00e-04 	 Train score: 0.4101 	 Cross val score: 0.4009 	 Nonzero coef: 6
Parameter: 6.00e-04 	 Train score: 0.4018 	 Cross val score: 0.3920 	 Nonzero coef: 6
Parameter: 7.00e-04 	 Train score: 0.3919 	 Cross val score: 0.3816 	 Nonzero coef: 6
Parameter: 8.00e-04 	 Train score: 0.3805 	 Cross val score: 0.3696 	 Nonzero coef: 6
Parameter: 1.00e-03 	 Train score: 0.3531 	 Cross val score: 0.3408 	 Nonzero coef: 5
Parameter: 2.00e-03 	 Train score: 0.3094 	 Cross val score: 0.3039 	 Nonzero coef: 4
Parameter: 5.00e-03 	 Train score: 0.1680 	 Cross val 

### 1.3 Active

In [90]:
y_index = 0

In [91]:
lr = linear_model.LinearRegression()
cross_results = cross_validate(lr, x, y[:,y_index], cv=group_split, groups=groups, scoring='r2', return_train_score=True)
print("Train score: %.4f \t Cross val score: %.4f \t " % 
          (cross_results['train_score'].mean(), cross_results['test_score'].mean()))

Train score: 0.4563 	 Cross val score: 0.4463 	 


In [92]:
# Lasso
for a in (1e-4)*np.array([0.1,1,2,3,4,5,6,7,8,10,20,50]):
    lasso = linear_model.Lasso(alpha=a)
    cross_results = cross_validate(lasso, x, y[:,y_index], cv=group_split, groups=groups, scoring='r2', return_train_score=True, return_estimator=True)
    nz = 0
    for m in cross_results['estimator']:
        nz += sum(m.coef_ != 0)
    nz /= 5
    
    print("Parameter: %.2e \t Train score: %.4f \t Cross val score: %.4f \t Nonzero coef: %d" % 
          (a, cross_results['train_score'].mean(), cross_results['test_score'].mean(), nz))

Parameter: 1.00e-05 	 Train score: 0.4563 	 Cross val score: 0.4463 	 Nonzero coef: 9
Parameter: 1.00e-04 	 Train score: 0.4558 	 Cross val score: 0.4461 	 Nonzero coef: 8
Parameter: 2.00e-04 	 Train score: 0.4544 	 Cross val score: 0.4448 	 Nonzero coef: 7
Parameter: 3.00e-04 	 Train score: 0.4525 	 Cross val score: 0.4431 	 Nonzero coef: 6
Parameter: 4.00e-04 	 Train score: 0.4500 	 Cross val score: 0.4409 	 Nonzero coef: 6
Parameter: 5.00e-04 	 Train score: 0.4467 	 Cross val score: 0.4380 	 Nonzero coef: 6
Parameter: 6.00e-04 	 Train score: 0.4430 	 Cross val score: 0.4346 	 Nonzero coef: 6
Parameter: 7.00e-04 	 Train score: 0.4387 	 Cross val score: 0.4310 	 Nonzero coef: 6
Parameter: 8.00e-04 	 Train score: 0.4338 	 Cross val score: 0.4268 	 Nonzero coef: 5
Parameter: 1.00e-03 	 Train score: 0.4231 	 Cross val score: 0.4168 	 Nonzero coef: 5
Parameter: 2.00e-03 	 Train score: 0.3410 	 Cross val score: 0.3354 	 Nonzero coef: 5
Parameter: 5.00e-03 	 Train score: 0.2638 	 Cross val 

# 2. MNL for mode choice

In [5]:
from util_aggregate_models import mnl_torch

# dataloader and model definition

lr_list = [0.01,0.1]
wd_list = [0.001, 0.0001]

results = {}
for i in range(5):
    
    train_filter = groups != i
    test_filter = groups == i
    x_train = x[train_filter]
    y_train = y[train_filter]
    x_test = x[test_filter]
    y_test = y[test_filter]
    sst_train = np.sum(np.power(y_train - np.mean(y_train, axis=0), 2), axis=0)
    sst_test = np.sum(np.power(y_test - np.mean(y_test, axis=0), 2), axis=0)
    
    trainset = SurveyDataset(torch.tensor(x_train,  dtype=torch.float), torch.tensor(y_train, dtype=torch.float))
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)

    testset = SurveyDataset(torch.tensor(x_test, dtype=torch.float), torch.tensor(y_test, dtype=torch.float))
    testloader = DataLoader(testset, batch_size=len(testset), shuffle=False)

    ret_dict = mnl_torch(trainloader, testloader, x_train.shape[-1], sst_train, sst_test, lr_list=lr_list, wd_list=wd_list)
    
    results[i] = ret_dict
    

[lr: 1.00e-02, wd: 1.00e-03]
Early stopping at epoch 730
[epoch: 725] Train KL loss: 0.153 Train R2 score: 0.415 0.498 -0.025 0.377 
[epoch: 725] Test KL loss: 0.152 Test R2 score: 0.434 0.520 -0.004 0.417 

[lr: 1.00e-02, wd: 1.00e-04]
Early stopping at epoch 720
[epoch: 715] Train KL loss: 0.153 Train R2 score: 0.418 0.507 -0.020 0.379 
[epoch: 715] Test KL loss: 0.152 Test R2 score: 0.441 0.527 -0.004 0.408 

[lr: 1.00e-01, wd: 1.00e-03]
Early stopping at epoch 250
[epoch: 245] Train KL loss: 0.147 Train R2 score: 0.444 0.540 -0.019 0.417 
[epoch: 245] Test KL loss: 0.148 Test R2 score: 0.454 0.550 -0.014 0.440 

[lr: 1.00e-01, wd: 1.00e-04]
Early stopping at epoch 225
[epoch: 220] Train KL loss: 0.147 Train R2 score: 0.446 0.543 -0.015 0.414 
[epoch: 220] Test KL loss: 0.148 Test R2 score: 0.459 0.554 -0.012 0.436 

[lr: 1.00e-02, wd: 1.00e-03]
Early stopping at epoch 710
[epoch: 705] Train KL loss: 0.150 Train R2 score: 0.429 0.510 -0.025 0.397 
[epoch: 705] Test KL loss: 0.163 Te

In [6]:
df = []

for (lr, wd) in itertools.product(lr_list, wd_list):

    new = []
    
    for i in range(5):
        new2 = []
        for k,v in results[i][(lr,wd)].items():
            new2.append(results[i][(lr,wd)][k]) 
        new.append(new2)
        
    new = np.array(new) 
    
    df.append([lr] + [wd] + list(np.mean(new, axis=0)))

pd.DataFrame(np.array(df), columns = ['learning_rate','weight_decay','train_kl_loss','test_kl_loss','train_r2_auto','train_r2_active','train_r2_pt',
                                     'test_r2_auto','test_r2_active','test_r2_pt'])

,learning_rate,weight_decay,train_kl_loss,test_kl_loss,train_r2_auto,train_r2_active,train_r2_pt,test_r2_auto,test_r2_active,test_r2_pt
0,0.01,0.0010,0.152954,0.154548,0.506089,0.420090,0.387202,0.497135,0.413614,0.374713
1,0.01,0.0001,0.152663,0.154859,0.506853,0.420345,0.390973,0.498612,0.407012,0.375048
2,0.10,0.0010,0.146541,0.148759,0.543257,0.448255,0.423402,0.534377,0.438627,0.405779
3,0.10,0.0001,0.146444,0.148644,0.544200,0.448406,0.423962,0.534984,0.439912,0.407085
